# Specific Heat in Diamond and Lead

We assume to have a material with $3N$ phonon modes, of Einstein frequency $\omega_E$.


## A. System Partition Function

For some solid of N atoms there will be 3N normal modes. Each normal mode, which can be represented classically as a wave of distortion in the lattice of the material, gives rise to a quantum of sound- a phonon of characteristic frequency $\omega_i$ and energy $\hbar \omega_i$. The quantum state of a solid (at low enough temperatures) can be modelled as a volume containing a gas of non-interacting phonons. The partition function for a mode:

$\displaystyle Q_N (V, T) = \sum_n e^{-\beta E_n}$.


For some state which has $n_i$ phonons of type i (i.e. of characteristic frequency $\omega_i$), the total energy of the state is:

$\displaystyle E\{n_i\} = \sum_{i=1}^{3N}n_i \hbar \omega_i$,

and so:

$\displaystyle Q_N (V, T) = \sum_{i=1}^{3N} e^{-\beta E_i}$.


Since we assume that $N \to \infty$, and using the **important assumption** that for an Einstein model all characteristic frequencies are taken to be equal, we can evaluate the expression with an infinite-term geometric progression to obtain:

$Q_N (V, T) = \frac{1}{1 - \exp({-\beta \hbar\omega_E})}$.

Since we assumed that the phonons are independent, the full partition function can be written as a product of the partition functions of nodes:

$\displaystyle Q = \prod^{3N}_{i = 1} Q_N$.

Therefore, the partition function for such a system is:

$\text{Q} = \prod\limits^{3N}_{i = 1} \frac{1}{1 - \exp(-\beta\hbar\omega_E)}$.



## B. Average Phonon Occupation Levels

The average occupation numbers are given by (Huang: 8.65 & 12.32):

$<n_i> := - \frac{1}{\beta} \frac{\partial}{\partial E_i} \log Q$.

Taking the log of the partition function:

$\log Q = $





## C. System Total Energy


## D. System Heat Capacity ($C_V$)

In [3]:
# Imports

import numpy as np
import scipy as sp
import requests
import pandas as pd
from io import StringIO
import pint #I shall certainly be importing a pint after this assignment is done :)

# !pip install pint


In [13]:
# Data Gathering for smooth sailing

diamond_url = 'https://raw.githubusercontent.com/diodeamy/StatMech/refs/heads/main/data/diamond.dat'
lead_url = 'https://raw.githubusercontent.com/diodeamy/StatMech/refs/heads/main/data/lead.dat'

diamond_resp = requests.get(diamond_url)
diamond_str = diamond_resp.text
diamond_io = StringIO(diamond_str)
diamond_df = pd.read_csv(diamond_io, comment='#', header=None, delim_whitespace=True)

lead_resp = requests.get(lead_url)
lead_str = lead_resp.text
lead_io = StringIO(lead_str)
lead_df = pd.read_csv(lead_io, header=None, delim_whitespace=True)


In [16]:
# Data Processing & Misc. for even smoother sailing?

ureg = pint.UnitRegistry()

## E. Diamond Einstein Temperature ($T_E$) (by fit)

In this section we look at the datafile for measurements of the specific heat of diamond in order to obtain the Einstein temperature of the system by fitting. 

## F. Diamond Debye Temperature ($T_D$) (by fit)

We wish to find the Debye temperature of diamond, by fitting the Debye model to the measurements of specific heat. We use a formulation such that the solution is more stable for small values of temperature. 

## G. Debye Model vs. Einstein Model Comparison (for diamond)

## H. Debye Model vs. Einstein Model Comparison (for lead)

### H.a. Einstein Temperature ($T_E$) (by fit) - lead

### H.b. Debye Temperature ($T_D$) (by fit) - lead

### H.c. Model Comparison - lead

## I. Diamond vs. Lead Comparison Discussion

## J. Fermi Temperature ($T_F$) (by fit) - lead

## K. Room-Temperature Entropy by Debye Function - diamond & lead